**Use GPU for better performance**


Credits:

* [qweasdd/manga-colorization-v2](https://github.com/qweasdd/manga-colorization-v2)

* [Keiser04/manga-colorization-v2](https://github.com/Keiser04/manga-colorization-v2)

* [xinntao/Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN/)

* [xinntao/Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN/)

* [ai-forever/Real-ESRGAN](https://github.com/ai-forever/Real-ESRGAN)

Colab By:

[Swakshan/AI-Manga-Colorizer](https://github.com/Swakshan/AI-Manga-Colorizer)

#Changelogs

---
> **Version 3**

* ++ Rewrite entire project
* ++ Merge all functions into one
* !! Added bulk chapter download
* !! Use manga code instead of url

---
> **Version 2**

* ++ Download using url
* ++ Image Compression

> **Version 2.5**

* !! change default generator.zip link
* !! display size after downloading project files
* ++ better file type checks

---
> **Version 1**

* Initial Release
---

#Utils

In [1]:
#@title Requirements
from IPython.display import clear_output

try:
  !pip install patool
  !pip install torchvision
  !pip install opencv-python
  !pip install python-telegram-bot
  !pip install filetype
  !pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git
  !pip install huggingface-hub==0.25.2
  clear_output()
except Exception as e:
  print(str(e))
print("DONE: Installed Requirements")

DONE: Installed Requirements


In [2]:
#@title Common

import os,requests,json,shutil
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from IPython.display import clear_output
from tqdm import tqdm
from zipfile import ZipFile



PROJECT_NAME = "manga-colorization-v2"

MANGA_FOLDER = f"/content/{PROJECT_NAME}/Manga"



def printLine():
  print("--------------------------")

def downloader(path,url,hdr={}):
  if os.path.exists(path):
    print(f"Exists: {path}")
    return True

  # print(f"Downloading {name}")
  r = requests.get(url,headers=hdr)
  total = int(r.headers.get('content-length', 0))
  if r.status_code == requests.codes.ok:
    with open(path, 'wb') as f,tqdm(
        desc=path,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    )as bar:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                size = f.write(chunk)
                bar.update(size)
    print(f"Downloaded: {path}")
    return
  else:
    print(f"Error Downloading : {path}")
    return False

def mkdir(path):
  try:
    os.makedirs(path)
    print(f"CREATED: {path}")
  except Exception as e:
    print(str(e))

def get_file_size_mb(file_path):
    # Get the size of the file in bytes
    size_in_bytes = os.path.getsize(file_path)
    # Convert bytes to megabytes (1 MB = 1024 * 1024 bytes)
    size_in_mb = size_in_bytes / (1024 * 1024)
    # print(size_in_mb)
    size_in_mb = round(size_in_mb,2)
    return f"{size_in_mb}MB"


def unzip(inp_dir,out_dir):
  try:
    with ZipFile(inp_dir, 'r') as zObject:
        zObject.extractall(path=out_dir)
    print(f"UNZIPPED: {out_dir}")
  except Exception as e:
    print(str(e))


def delete(path):
  if os.path.isdir(path):
    shutil.rmtree(path)
  else:
    os.remove(path)

In [3]:
#@title Download Project Files
import os

#@markdown Clone Keiser04/manga-colorization-v2
project = True # @param {type:"boolean"}
generator = True # @param {type:"boolean"}
generator_model_url = "https://drive.usercontent.google.com/download?id=1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t&export=download&confirm=t&uuid=cf1b41fa-03b3-49bf-8373-8f5d423c775d" # @param {type:"string"}
#@markdown ---
extractor = True # @param {type:"boolean"}
extractor_model_url = "https://drive.google.com/file/d/12cbNyJcCa1zI2EBz6nea3BXl21Fm73Bt/view?usp=sharing" # @param {type:"string"}
#@markdown ---
denoiser = True # @param {type:"boolean"}
denoiser_model_url = "https://drive.google.com/uc?export=download&id=161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3" # @param {type:"string"}
#@markdown ---

G_PATH = f"{PROJECT_NAME}/networks/"
D_PATH = f"{PROJECT_NAME}/denoising/models/"

done_str = ""
if project:
  !git clone https://github.com/Keiser04/manga-colorization-v2.git
  done_str+=" "+PROJECT_NAME+"\n"
if generator:
  mkdir(G_PATH)
  dPath = f"{G_PATH}generator.zip"
  downloader(dPath,generator_model_url)
  done_str=f"{done_str} generator.zip - ({get_file_size_mb(dPath)})\n"
if extractor:
  print(extractor_model_url)
  mkdir(G_PATH)
  dPath = f"{G_PATH}extractor.pth"
  downloader(dPath,extractor_model_url)
  done_str=f"{done_str} extractor.pth - ({get_file_size_mb(dPath)})\n"
if denoiser:
  mkdir(D_PATH)
  dPath = f"{D_PATH}net_rgb.pth"
  downloader(dPath,denoiser_model_url)
  done_str=f"{done_str} net_rgb.pth - ({get_file_size_mb(dPath)})\n"


clear_output()

printLine()
os.chdir(PROJECT_NAME)
mkdir(MANGA_FOLDER)
print("DONE:"+done_str)

--------------------------
CREATED: /content/manga-colorization-v2/Manga
DONE: manga-colorization-v2
 generator.zip - (122.99MB)
 extractor.pth - (0.09MB)
 net_rgb.pth - (3.28MB)



In [26]:
#@title Functions
import torch
from PIL import Image
from RealESRGAN import RealESRGAN
import matplotlib.pyplot as plt
import filetype
from io import BytesIO
import patoolib
from telegram import Bot
from telegram.error import TelegramError
from google.colab import drive
import PIL
import os
import glob
from colorizator import MangaColorizator
import zipfile
from pprint import pp

URL_MANGA_PILL = 'https://mangapill.com'
def downloadMangaPill(link):
    if URL_MANGA_PILL not in MANGA_CHAP_URL:
      raise Exception("Mangapill link only")
    hdr = {"Referer":f"{URL_MANGA_PILL}/"}
    # chp_name = chapter.text

    # Load the chapter
    c = requests.get(link)

    if c.status_code == requests.codes.ok:
      chapter_parser = BeautifulSoup(c.text, 'html.parser')
      title = chapter_parser.title
      title_split = title.text.split(" Chapter ")
      manga_name = title_split[0].title()
      CHAP_NAME = "Chapter "+title_split[1].replace(" - Mangapill",'')

      MANGA_PATH = MANGA_FOLDER+"/"+manga_name + '/' + CHAP_NAME
      mkdir(MANGA_PATH)
      MANGA_DETAILS = {'chap':CHAP_NAME,'manga':manga_name}
      pages = chapter_parser.find_all('img', attrs={'class': 'js-page'})

      for num,page in enumerate(pages):
        num+=1
        img_url = page['data-src']
        a = urlparse(img_url)
        fileName,ext = os.path.splitext(a.path)
        fileName = "{:05d}{ext}".format(num,ext=ext)
        down_path = MANGA_PATH+"/"+fileName
        downloader(down_path,img_url,hdr)
      clear_output()
      print(f"Downloaded at {MANGA_PATH} - {num} files")
      return manga_name,CHAP_NAME

def getMangaPillChapters(manga_code,start:str,count:int):
  link = f"{URL_MANGA_PILL}/manga/{manga_code}"
  c = requests.get(link)

  if c.status_code != requests.codes.ok:
    raise Exception("Manga Pill chapter not found")

  chapter_parser = BeautifulSoup(c.text, 'html.parser')
  chap_list = chapter_parser.find("div",{"id":"chapters"}).find_all("a")

  ch = []
  for chap in chap_list:
    chap_numb = chap.text.replace("Chapter ","")
    ch.append(chap_numb)

  ch = list(reversed(ch))
  s = ch.index(start)
  if s<0:
    raise Exception("Start chapter not found")
  rd = ch[s:s+count]
  return rd

#-----------------------------------
#-----------------------------------

CMP_FOLDER = "compress"
def image_compression(PATH,QUALITY):
  def compress(img_path,out_img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    buffer = BytesIO()
    img.save(buffer, "JPEG", quality=int(QUALITY))

    f = open(out_img_path, "wb")
    f.write(buffer.getbuffer())
    f.close()
    print(f"DONE: {out_img_path}")
    final_out = ""
  if os.path.isfile(PATH):
    name,ext = os.path.splitext(PATH)
    out_img_path = f"{name}_q{QUALITY}{ext}"
    compress(PATH,out_img_path)
    final_out = out_img_path

  else:
    out_path = os.path.join(PATH,CMP_FOLDER)
    mkdir(out_path)
    final_out = out_path
    # print(out_path)
    for img_name in os.listdir(PATH):
      img_path = os.path.join(PATH,img_name)
      if os.path.isfile(img_path):
        out_img_path = os.path.join(out_path,img_name)
        if not filetype.is_image(img_path):continue
        compress(img_path,out_img_path)
        # clear_output()

  clear_output()
  print(f"DONE: {final_out}")

#-----------------------------------
#-----------------------------------

def buildCBZ(IMAGES_FOLDER,ZIP_FILENAME):
    output_path = os.path.join(IMAGES_FOLDER, ZIP_FILENAME)
    if os.path.exists(output_path):
      ch = input(f"{ZIP_FILENAME}\nalready exists,Delete the existsing one ? (1/0): ")
      if (int(ch)):
        os.remove(output_path)
      else:
        return

    image_files = glob.glob(os.path.join(IMAGES_FOLDER, '*.jpeg'))

    # # Create the zip file
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Directly add the entire directory
        for img in image_files:
          zipf.write(img)
    printLine()
    rd = {"path":output_path,"count":len(image_files),"size": get_file_size_mb(output_path)}
    pp(rd)
    return rd

#-----------------------------------
#-----------------------------------

async def sendTELE(BOT_TOKEN,CHANNEL_ID,DOC,CAPTION):
  if len(BOT_TOKEN) and len(CHANNEL_ID) and len(DOC):
    text = CAPTION
    zip_filename = DOC
    print(f"SENDING: {zip_filename}")

    bot = Bot(token=BOT_TOKEN)
    try:
      await bot.send_document(chat_id=CHANNEL_ID, document=open(zip_filename, 'rb'),caption=text)
      print(f'SENT: {text}')
    except TelegramError as e:
        print(f'Failed to send document: {e}')

  else:
    print("Check token and channel id and doc")

#-----------------------------------
#-----------------------------------

def send2Drive(SRC_FILE,DEST_FOLDER):
  DRIVE = "/content/drive/"
  if not os.path.exists(DRIVE):
    drive.mount(DRIVE)

  DRIVE_MANGA = os.path.join('/MyDrive/Manga/',DEST_FOLDER)
  if not DRIVE in DEST_FOLDER:
    DRIVE_MANGA = os.path.join(DRIVE,DRIVE_MANGA)
  mkdir(DRIVE_MANGA)

  if os.path.exists(SRC_FILE):
    shutil.copy(SRC_FILE,DRIVE_MANGA)
    print(f"COPIED: to {DRIVE_MANGA}")
  else:
    print(f"FAILED: {SRC_FILE} doesnt exists")

#-----------------------------------
#-----------------------------------

CLR_FOLDER = "colour"
class Colorize():
  def __init__(self,MANGA_PATH,DEVICE,SIZE, DENOISE, DENOISE_SIGMA,GENERATOR,EXTRACTOR):
    self.MANGA_PATH= MANGA_PATH
    self.DEVICE= DEVICE
    self.SIZE= SIZE
    self.DENOISE= DENOISE
    self.DENOISE_SIGMA= DENOISE_SIGMA
    self.GENERATOR= GENERATOR
    self.EXTRACTOR= EXTRACTOR

  def convert_to_bw(self,img_path):
      color_image = Image.open(img_path)
      bw = color_image.convert('L')
      bw.save(img_path)

  def process_image(self,image, colorizator):
      colorizator.set_image(image, self.SIZE, self.DENOISE, self.DENOISE_SIGMA)
      return colorizator.colorize()

  def color_it(self,image_path,save_path):
    self.convert_to_bw(image_path)
    image = plt.imread(image_path)
    colorizer = MangaColorizator(self.DEVICE,self.GENERATOR,self.EXTRACTOR)
    colorization = self.process_image(image, colorizer)
    plt.imsave(save_path, colorization)

  def colourize_process(self):
    try:
      temp_name = CLR_FOLDER
      org_path = self.MANGA_PATH
      if os.path.isdir(org_path):
          SAVE_PATH = os.path.join(org_path,temp_name)
          mkdir(SAVE_PATH)
          num = 0
          for img in os.listdir(org_path):
            image_path = os.path.join(org_path,img)
            save_path = os.path.join(SAVE_PATH,img)
            if not os.path.isfile(image_path):continue
            if not filetype.is_image(image_path):continue
            if f"_{temp_name}" in img:continue
            num+=1
            self.color_it(image_path,save_path)
            print(f"SAVED: {save_path}")
          clear_output()
          print(f"SAVED: {SAVE_PATH} - {num} files")

      elif os.path.isfile(org_path):
        fileName,ext = os.path.splitext(org_path)
        save_path = f"{fileName}_{temp_name}{ext}"
        self.color_it(org_path,save_path)
        print(f"SAVED: {save_path}")
      else:
        print("Check the path provided..")
    except Exception as e:
      print(str(e))

#-----------------------------------
#-----------------------------------
UPSC_FOLDER = "upscaled"
class Upscaler():
  def __init__(self,device="cpu",scale=2,weight="RealESRGAN_x4.pth"):
    self.scale = scale
    device = torch.device(device)
    self.model = RealESRGAN(device, scale=self.scale)
    self.model.load_weights(f'weights/{weight}', download=True)

  def __upscale(self,image_path):
    og_folder,og_filename = os.path.split(image_path)
    out_folder = os.path.join(og_folder,UPSC_FOLDER)
    if not os.path.exists(out_folder): mkdir(out_folder)
    name,ext = os.path.splitext(og_filename)
    save_filename = f"{name}_x{self.scale}{ext}"
    save_path = os.path.join(out_folder,save_filename)

    image = Image.open(image_path).convert('RGB')
    sr_image = self.model.predict(image)
    sr_image.save(save_path)

  def process(self,path):
    if os.path.isfile(path):
      self.__upscale(path)
    else:
      print(path)
      for img in os.listdir(path):
        file_path = os.path.join(path,img)
        if os.path.isfile(file_path) and filetype.is_image(file_path):
          self.__upscale(file_path)

# Main

In [ ]:
GPU = True# @param {type:"boolean"}
#@markdown MangaPill details
MANGA_CODE = "7529" # @param {type:"string"}
CHAP_CODE = "66" # @param {type:"string"}
#@markdown
#@markdown ---
#@markdown default 0
NUMBER_OF_CHAPTERS = 0 # @param {"type":"integer"}
#@markdown ---
#@markdown Colourize Config
GENERATOR = "networks/generator.zip" # @param {type:"string"}
EXTRACTOR = "networks/extractor.pth" # @param {type:"string"}
DENOISE = True# @param {type:"boolean"}
DENOISE_SIGMA = 25# @param {type:"integer"}
#@markdown default SIZE_FACTOR = 18
SIZE_FACTOR = 20# @param {type:"integer"}
#@markdown ---
#@markdown ---
#@markdown Upscaler
#@markdown default "SCALE_FACTOR" = 2
SCALE_FACTOR = "2" # @param ["2", "3", "4"]
UPSCALER_WEIGHT = "RealESRGAN_x4.pth" # @param {type:"string"}
#@markdown ---
#@markdown ---
#@markdown Compressor
QUALITY = 60 # @param {type:"integer"}
#@markdown ---
#@markdown ---
COPY_TO_DRIVE = False# @param {type:"boolean"}
#@markdown ---
#@markdown ---
SEND_TO_TELEGRAM = True# @param {type:"boolean"}
TELE_CHANNEL_ID = "" # @param {type:"string"}
#@markdown Add TELE_BOT_TOKEN in secrets and add -100 if your channel is private

from google.colab import userdata
BOT_TOKEN = userdata.get('TELE_BOT_TOKEN')

DEVICE = 'cpu'
if GPU:
  DEVICE = 'cuda'

# CHAPS is an array
# if NUMBER_OF_CHAPTERS is 0 then download only CHAP_CODE
# else get the list of chaps
CHAPS = [CHAP_CODE] if NUMBER_OF_CHAPTERS==0 else getMangaPillChapters(MANGA_CODE,CHAP_CODE,NUMBER_OF_CHAPTERS)

final_out = []
for CHAP in CHAPS:
  clear_output()
  cs = CHAP.split(".")
  ec = "000"
  if len(cs)>1:
    ec = int(cs[1])*100
  sc = 10000+int(cs[0])
  MANGA_CHAP_URL =f"https://mangapill.com/chapters/{MANGA_CODE}-{sc}{ec}"

  MANGA_NAME,CHAP_NAME = downloadMangaPill(MANGA_CHAP_URL)
  MANGA_PATH = f'{MANGA_FOLDER}/{MANGA_NAME}/{CHAP_NAME}'

  SIZE = SIZE_FACTOR*32
  c = Colorize(MANGA_PATH,DEVICE,SIZE, DENOISE, DENOISE_SIGMA,GENERATOR,EXTRACTOR)
  c.colourize_process()
  MANGA_PATH = f'{MANGA_PATH}/{CLR_FOLDER}'


  up = Upscaler(DEVICE,int(SCALE_FACTOR),UPSCALER_WEIGHT)
  up.process(MANGA_PATH)
  MANGA_PATH = f'{MANGA_PATH}/{UPSC_FOLDER}'


  image_compression(MANGA_PATH,QUALITY)
  IMAGES_FOLDER = f'{MANGA_PATH}/{CMP_FOLDER}'


  ZIP_FILENAME = f'{CHAP_NAME}.cbz'
  o = buildCBZ(IMAGES_FOLDER,ZIP_FILENAME)
  final_out.append(o)

  DOC = f'{MANGA_PATH}/{CMP_FOLDER}/{ZIP_FILENAME}'
  if COPY_TO_DRIVE:
    send2Drive(DOC,MANGA_NAME)

  if SEND_TO_TELEGRAM:
    CAPTION = f'{MANGA_NAME} - {CHAP_NAME}'
    await sendTELE(BOT_TOKEN,TELE_CHANNEL_ID,DOC,CAPTION)

clear_output()
print(f"Downloaded : {CHAPS}")
pp(final_out)

In [28]:
import shutil
# @title remove folder
folder_path = "/content/manga-colorization-v2/Manga/Kagurabachi" # @param {type:"string"}

shutil.rmtree(folder_path)